In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:

import string
import numpy as np
import random

ALPHABET = string.ascii_uppercase


# Utility Functions


def clean_text(text):
    "Convert text to uppercase and keep only alphabetic characters (A–Z). Removes spaces, hyphens, and punctuation."
    
    return "".join(ch for ch in text.upper() if ch in ALPHABET)


def mod_inverse(a, m):
    "Find modular inverse of a under modulo m."
    a = a % m
    for i in range(1, m):
        if (a * i) % m == 1:
            return i
    raise ValueError("Modular inverse does not exist")

# Activity Logging 


def log_activity(cipher, operation, input_text, output_text):
    """
    Logs encryption/decryption activity to a file.
    """
    with open("crypto_activity_log.txt", "a") as f:
        f.write(f"{cipher} | {operation} | {input_text} -> {output_text}\n")


# Caesar Cipher


def caesar_encrypt(text, key):
    text = clean_text(text)
    result = ""
    for ch in text:
        result += ALPHABET[(ALPHABET.index(ch) + key) % 26]
    return result

def caesar_decrypt(text, key):
    return caesar_encrypt(text, -key)


# Affine Cipher

def affine_encrypt(text, a, b):
    text = clean_text(text)
    result = ""
    for ch in text:
        result += ALPHABET[(a * ALPHABET.index(ch) + b) % 26]
    return result

def affine_decrypt(text, a, b):
    text = clean_text(text)
    a_inv = mod_inverse(a, 26)
    result = ""
    for ch in text:
        result += ALPHABET[(a_inv * (ALPHABET.index(ch) - b)) % 26]
    return result


# Playfair Cipher


def generate_playfair_matrix(key):
    key = clean_text(key).replace("J", "I")
    matrix = []
    for ch in key:
        if ch not in matrix:
            matrix.append(ch)
    for ch in ALPHABET:
        if ch not in matrix and ch != "J":
            matrix.append(ch)
    return [matrix[i*5:(i+1)*5] for i in range(5)]

def find_position(matrix, ch):
    for i in range(5):
        for j in range(5):
            if matrix[i][j] == ch:
                return i, j

def prepare_playfair_text(text):
    text = clean_text(text).replace("J", "I")
    pairs = []
    i = 0
    while i < len(text):
        a = text[i]
        b = text[i+1] if i+1 < len(text) and text[i] != text[i+1] else "X"
        pairs.append(a + b)
        i += 2 if b != "X" else 1
    return pairs

def playfair_encrypt(text, key):
    matrix = generate_playfair_matrix(key)
    pairs = prepare_playfair_text(text)
    result = ""
    for pair in pairs:
        r1, c1 = find_position(matrix, pair[0])
        r2, c2 = find_position(matrix, pair[1])
        if r1 == r2:
            result += matrix[r1][(c1+1)%5] + matrix[r2][(c2+1)%5]
        elif c1 == c2:
            result += matrix[(r1+1)%5][c1] + matrix[(r2+1)%5][c2]
        else:
            result += matrix[r1][c2] + matrix[r2][c1]
    return result

def playfair_decrypt(text, key):
    matrix = generate_playfair_matrix(key)
    result = ""
    for i in range(0, len(text), 2):
        r1, c1 = find_position(matrix, text[i])
        r2, c2 = find_position(matrix, text[i+1])
        if r1 == r2:
            result += matrix[r1][(c1-1)%5] + matrix[r2][(c2-1)%5]
        elif c1 == c2:
            result += matrix[(r1-1)%5][c1] + matrix[(r2-1)%5][c2]
        else:
            result += matrix[r1][c2] + matrix[r2][c1]
    return result

# Hill Cipher Utilities 


def is_invertible(key):
    det = int(round(np.linalg.det(key))) % 26
    try:
        mod_inverse(det, 26)
        return True
    except:
        return False

def generate_valid_hill_key():
    while True:
        key = np.array([[random.randint(0,25), random.randint(0,25)],
                        [random.randint(0,25), random.randint(0,25)]])
        if is_invertible(key):
            return key


# Hill Cipher (2x2)


def hill_encrypt(text, key):
    text = clean_text(text)
    if len(text) % 2 != 0:
        text += "X"
    result = ""
    for i in range(0, len(text), 2):
        vec = np.array([[ALPHABET.index(text[i])],
                        [ALPHABET.index(text[i+1])]])
        enc = np.dot(key, vec) % 26
        result += ALPHABET[int(enc[0])] + ALPHABET[int(enc[1])]
    return result

def hill_decrypt(text, key):
    det = int(round(np.linalg.det(key))) % 26
    det_inv = mod_inverse(det, 26)
    adj = np.array([[key[1][1], -key[0][1]],
                    [-key[1][0], key[0][0]]])
    inv_key = (det_inv * adj) % 26
    return hill_encrypt(text, inv_key)

# Hill Cipher Known-Plaintext Attack


def hill_crack(plaintext, ciphertext):
    plaintext = clean_text(plaintext)
    ciphertext = clean_text(ciphertext)

    P = np.array([[ALPHABET.index(plaintext[0]), ALPHABET.index(plaintext[2])],
                  [ALPHABET.index(plaintext[1]), ALPHABET.index(plaintext[3])]])

    C = np.array([[ALPHABET.index(ciphertext[0]), ALPHABET.index(ciphertext[2])],
                  [ALPHABET.index(ciphertext[1]), ALPHABET.index(ciphertext[3])]])

    det = int(round(np.linalg.det(P))) % 26
    det_inv = mod_inverse(det, 26)

    adj = np.array([[P[1][1], -P[0][1]],
                    [-P[1][0], P[0][0]]])

    P_inv = (det_inv * adj) % 26
    return (np.dot(C, P_inv)) % 26






In [10]:

# Main Menu


def launch_crypto_tool():
    while True:
        print("\nClassic Cryptography Tool (Kaggle)")
        print("1. Caesar Cipher")
        print("2. Affine Cipher")
        print("3. Playfair Cipher")
        print("4. Hill Cipher (2x2)")
        print("5. Hill Cipher Known-Plaintext Attack")
        print("0. Exit")

        choice = input("Enter your choice: ")

        if choice == "1":
            key = int(input("Enter shift key: "))
            text = input("Enter text: ")
            op = input("E = Encrypt, D = Decrypt: ").upper()
            result = caesar_encrypt(text, key) if op == "E" else caesar_decrypt(text, key)
            print("Result:", result)
            log_activity("Caesar", "Encrypt" if op == "E" else "Decrypt", text, result)

        elif choice == "2":
            a = int(input("Enter a (coprime with 26): "))
            b = int(input("Enter b: "))
            text = input("Enter text: ")
            op = input("E = Encrypt, D = Decrypt: ").upper()
            result = affine_encrypt(text, a, b) if op == "E" else affine_decrypt(text, a, b)
            print("Result:", result)
            log_activity("Affine", "Encrypt" if op == "E" else "Decrypt", text, result)

        elif choice == "3":
            key = input("Enter Playfair key: ")
            text = input("Enter text: ")
            op = input("E = Encrypt, D = Decrypt: ").upper()
            result = playfair_encrypt(text, key) if op == "E" else playfair_decrypt(text, key)
            print("Result:", result)
            log_activity("Playfair", "Encrypt" if op == "E" else "Decrypt", text, result)

        elif choice == "4":
            print("1. Enter key manually")
            print("2. Generate secure random key")
            opt = input("Choose option: ")

            if opt == "2":
                key = generate_valid_hill_key()
                print("Generated Key:\n", key)
            else:
                a = int(input("a = "))
                b = int(input("b = "))
                c = int(input("c = "))
                d = int(input("d = "))
                key = np.array([[a, b], [c, d]])

                if not is_invertible(key):
                    print("Invalid Hill key! Determinant not invertible.")
                    continue

            text = input("Enter text: ")
            op = input("E = Encrypt, D = Decrypt: ").upper()
            result = hill_encrypt(text, key) if op == "E" else hill_decrypt(text, key)
            print("Result:", result)
            log_activity("Hill", "Encrypt" if op == "E" else "Decrypt", text, result)

        elif choice == "5":
            pt = input("Enter 4-letter plaintext: ")
            ct = input("Enter 4-letter ciphertext: ")
            recovered_key = hill_crack(pt, ct)
            print("Recovered Key Matrix:")
            print(recovered_key)
            log_activity("Hill Cracker", "Key Recovery", pt, recovered_key)

        elif choice == "0":
            print("Exiting tool.")
            break

        else:
            print("Invalid choice.")


In [ ]:
launch_crypto_tool()



Classic Cryptography Tool (Kaggle)
1. Caesar Cipher
2. Affine Cipher
3. Playfair Cipher
4. Hill Cipher (2x2)
5. Hill Cipher Known-Plaintext Attack
0. Exit


Enter your choice:  1
Enter shift key:  3
Enter text:  I like juice
E = Encrypt, D = Decrypt:  E


Result: LOLNHMXLFH

Classic Cryptography Tool (Kaggle)
1. Caesar Cipher
2. Affine Cipher
3. Playfair Cipher
4. Hill Cipher (2x2)
5. Hill Cipher Known-Plaintext Attack
0. Exit
